In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from fastai.tabular.core import add_datepart

! pip install h3
import h3

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download ealtman2019/credit-card-transactions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 263M/263M [00:08<00:00, 38.4MB/s]
100% 263M/263M [00:08<00:00, 33.5MB/s]


In [ ]:
import zipfile
!unzip /content/credit-card-transactions.zip


Archive:  /content/credit-card-transactions.zip
  inflating: User0_credit_card_transactions.csv  
  inflating: credit_card_transactions-ibm_v2.csv  
  inflating: sd254_cards.csv         
  inflating: sd254_users.csv         


upload the credit card transaction

In [ ]:
#Creating dataframe for the respective csv's

df_v2 = pd.read_csv("/content/credit_card_transactions-ibm_v2.csv")




reduce the data size by changing type to category

reduce the data size by changing binary Labels to 0 and 1 


In [ ]:
df_v2['Use Chip'] = df_v2['Use Chip'].astype('category')
df_v2['Use Chip'] = df_v2['Use Chip'].cat.rename_categories({'Swipe Transaction': 'S', 'Chip Transaction': 'Ch', 'Online Transaction': 'ON'})
df_v2['Is Fraud?']=df_v2['Is Fraud?'].replace({'No':0,'Yes':1}) 
df_v2['Is Fraud?']=df_v2['Is Fraud?'].astype(int)

treating the Error column by fill na by creating new categrory  no error
also tranformed the Error column to optimized version of 0 hot encoding 
and deleted the main Error column 

In [ ]:
df_v2['Errors?']=df_v2['Errors?'].fillna('No Errors')
df_v2['Bad PIN'] = df_v2['Errors?'].apply(lambda x: 1 if 'Bad PIN' in x else 0)
df_v2['Technical Glitch'] = df_v2['Errors?'].apply(lambda x: 1 if 'Technical Glitch' in x else 0)
df_v2['Bad Card Number'] = df_v2['Errors?'].apply(lambda x: 1 if 'Bad Card Number' in x else 0)
df_v2['Bad CVV'] = df_v2['Errors?'].apply(lambda x: 1 if 'Bad CVV' in x else 0)
df_v2['Bad Expiration'] = df_v2['Errors?'].apply(lambda x: 1 if 'Bad Expiration' in x else 0)
df_v2['Bad Zipcode'] = df_v2['Errors?'].apply(lambda x: 1 if 'Bad Zipcode' in x else 0)
df_v2['Errors'] = df_v2['Errors?'].apply(lambda x: 0 if 'No Errors' in x else 1)
df_v2['Insufficient Balance'] = df_v2['Errors?'].apply(lambda x: 1 if 'Insufficient Balance' in x else 0)
del df_v2['Errors?']


treated the empty value  for Merchant state  as all values are empty whenthe the transaction to online merchant

also removed the $ from the amount column 
also removed the space in the column name 


In [ ]:
df_v2['Merchant State']=df_v2['Merchant State'].fillna('ONLINE')
df_v2["Amount"] = df_v2["Amount"].str.replace('[\$\,\.]','', regex=True).astype('float64')
df_v2.columns = [c.replace(' ', '_') for c in df_v2.columns]

creating new column for if the transaction international 0  and 1 if local

treating the missing column in zipcode by 0 if the transaction online and 99999 if international

In [ ]:
pd.options.mode.chained_assignment = None
df_v2.Zip[df_v2.Merchant_State=="ONLINE"] = 0
S = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY','AA','AE','ONLINE','AP']
df_v2['InternationalT'] = df_v2['Merchant_State'].apply(lambda x: 1 if x in S else 0)
df_v2['Zip']=df_v2['Zip'].fillna(99999)



created  categrorical feature based on the time of the transaction at the day 

add_datepart Helper function that adds columns relevant to a date in the column for refrence https://docs.fast.ai/tabular.core.html
also created hour of the day  and month of the year cycle feature to capture the if there any cyclical relation 



In [ ]:
df_v2['Timec'] = pd.to_datetime(df_v2['Time'],format= '%H:%M' )
df_v2['Timec'] = (df_v2['Timec'].dt.hour % 24 + 4) // 4
df_v2['Timec'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)
df_v2['Date']=pd.to_datetime(df_v2['Year'].apply(str)+'-'+ df_v2['Month'].apply(str)+'-'+ df_v2['Day'].apply(str)+'-'+ df_v2['Time'], format='%Y-%m-%d-%H:%M')
add_datepart (df_v2, 'Date', prefix='Tran', drop=False)
del df_v2['Year']
del df_v2['Month']
del df_v2['Day']
del df_v2['Merchant_City']

df_v2['Hour'] = df_v2['Date'].dt.hour + ((df_v2['Date'].dt.minute)/60)
df_v2['hr_sin'] = np.sin(df_v2.Hour*(2.*np.pi/24))
df_v2['hr_cos'] = np.cos(df_v2.Hour*(2.*np.pi/24))
df_v2['mnth_sin'] = np.sin((df_v2.TranMonth-1)*(2.*np.pi/12))
df_v2['mnth_cos'] = np.cos((df_v2.TranMonth-1)*(2.*np.pi/12))
del df_v2['Time']

frequency enciding for high cardinality categorical features 

In [ ]:
enc_MN = (df_v2.groupby('Merchant_Name').size()) / len(df_v2)
df_v2['enc_MN'] = df_v2['Merchant_Name'].apply(lambda x : enc_MN[x])
del df_v2['Merchant_Name']
del enc_MN
enc_MS = (df_v2.groupby('Merchant_State').size()) / len(df_v2)
df_v2['enc_MS'] = df_v2['Merchant_State'].apply(lambda x : enc_MS[x])
del df_v2['Merchant_State']
del enc_MS
enc_UI = (df_v2.groupby('User').size()) / len(df_v2)
df_v2['enc_UI'] = df_v2['User'].apply(lambda x : enc_UI[x])
del enc_UI

reduce dat size by changing boolean features to int 0 or 1

In [ ]:
for col in df_v2:
  if df_v2[col].dtype == bool  :
    df_v2[col] = df_v2[col].astype(int)

treating the  users and cards df same as transaction data and do some feature engineering 

In [ ]:
df_users = pd.read_csv("/content/sd254_users.csv")
df_cards = pd.read_csv("/content/sd254_cards.csv")
df_cards['Card Brand'] = df_cards['Card Brand'].astype('category')
df_cards['Card Brand'] = df_cards['Card Brand'].cat.rename_categories({'Mastercard': 'M', 'Visa': 'V', 'Amex': 'A', 'Discover': 'D'})
df_cards['Card Type'] = df_cards['Card Type'].astype('category')
df_cards['Card Type'] = df_cards['Card Type'].cat.rename_categories({'Debit': 'D', 'Credit': 'C', 'Debit (Prepaid)': 'DP'})
df_cards['Has Chip'] = df_cards['Has Chip'].astype('category')
df_cards['Has Chip'] = df_cards['Has Chip'].cat.rename_categories({'YES': 1, 'NO': 0})
df_cards['Card on Dark Web'] = df_cards['Card on Dark Web'].astype('category')
df_cards['Card on Dark Web'] = df_cards['Card on Dark Web'].cat.rename_categories({'Yes': 1, 'No': 0})
df_cards["Credit Limit"] = df_cards["Credit Limit"].str.replace('[\$\,\.]','', regex=True).astype('int64')
df_cards['Expires'] = pd.to_datetime(df_cards['Expires'],format='%m/%Y')

df_cards['Expires_year']=df_cards['Expires'].dt.year 
df_cards['Expires_month']=df_cards['Expires'].dt.month 
df_cards['Acct Open Date'] = pd.to_datetime(df_cards['Acct Open Date'],format='%m/%Y')

df_cards['Acct Open Date_year']=df_cards['Acct Open Date'].dt.year 
df_cards['Acct Open Date_month']=df_cards['Acct Open Date'].dt.month 

df_cards['frequent pin']=df_cards['Year PIN last Changed']-df_cards['Acct Open Date_year']
df_cards['frequecy change cardm']=( df_cards['Expires']- df_cards['Acct Open Date'] )  / np.timedelta64(1,'M')
df_cards['frequecy change cardm'] = df_cards['frequecy change cardm'].astype(int)
df_cards.drop(['Card Number', 'Expires', 'Acct Open Date', 'Year PIN last Changed','Card on Dark Web','CVV'], axis=1, inplace=True)

In [ ]:
df_users["Per Capita Income - Zipcode"] = df_users["Per Capita Income - Zipcode"].str.replace('[\$\,\.]','', regex=True).astype('int64')
df_users["Yearly Income - Person"] = df_users["Yearly Income - Person"].str.replace('[\$\,\.]','', regex=True).astype('int64')
df_users["Total Debt"] = df_users["Total Debt"].str.replace('[\$\,\.]','', regex=True).astype('int64')
df_users['Apartment']=df_users['Apartment'].fillna(0)
df_users['Apartment'] = df_users['Apartment'].apply(lambda x:x if x == 0 else 1)
df_users['Apartment']=df_users['Apartment'].astype('int64')
df_users['Gender'] = df_users['Gender'].apply(lambda x:1 if x == "Female" else 0)


calculate distance between the customer and the merchant as KM also provided for any online transaction this is 0 
and for all international transaction 10000 KM

In [ ]:
df_v2 = pd.merge(df_v2,df_users [['Zipcode','Latitude','Longitude']], left_on='User',right_index=True ,how='left')
df_v2.Zip[df_v2.Zip==0]=df_v2['Zipcode']
df_users.drop(['Zipcode', 'Latitude' , 'Longitude','Address','City','State','Person'], axis=1, inplace=True)
df_zip = pd.read_csv("/content/drive/MyDrive/DataMining/US.txt", sep="\t",header=None)
df_zip.drop(df_zip.columns[[0,2,3,4,5,6,7,8,11]], axis=1, inplace=True)
df_zip = df_zip.set_axis(['ZIP', 'Lat', 'Long',], axis=1, inplace=False)
df_v2['Zip'] = df_v2['Zip'].astype(int)
df_zip.sort_values(by='ZIP', inplace=True)
df_v2.sort_values(by='Zip', inplace=True)
df_v2 = pd.merge_asof( df_v2,df_zip, left_on='Zip',right_on='ZIP',direction="nearest")
del df_zip
del df_v2['ZIP']
del df_v2['Zipcode']
del df_v2['Zip']
result=[]
for row in zip(df_v2['Latitude'], df_v2['Longitude'],df_v2['Lat'],df_v2['Long']):
  result.append(h3.point_dist((row[0], row[1]), (row[2], row[3]), unit='km'))
df_v2['distance']=pd.Series(result)
del result
df_v2.distance[df_v2.InternationalT==0]=10000
del df_v2['Latitude']
del df_v2['Longitude']
del df_v2['Long']
del df_v2['Lat']

merge the transaction with cards dataset

In [ ]:
df_v2 = pd.merge(df_v2, df_cards,  how='left', left_on=['User','Card'], right_on = ['User','CARD INDEX'])
del df_v2['CARD INDEX']


merge also the users transaction 

In [ ]:
df_v2 = pd.merge(df_v2, df_users, left_on='User',right_index=True ,how='left')
df_v2.head(10)

,User,Card,Amount,Use_Chip,MCC,Is_Fraud?,Bad_PIN,Technical_Glitch,Bad_Card_Number,Bad_CVV,...,Retirement Age,Birth Year,Birth Month,Gender,Apartment,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,658,5,1475.0,S,4121,0,0,0,0,0,...,67,1968,7,1,0,55274,112695,35135,840,6
1,658,4,1525.0,S,4121,0,0,0,0,0,...,67,1968,7,1,0,55274,112695,35135,840,6
2,658,4,2009.0,S,4121,0,0,0,0,0,...,67,1968,7,1,0,55274,112695,35135,840,6
3,658,5,1643.0,S,4121,0,0,0,0,0,...,67,1968,7,1,0,55274,112695,35135,840,6
4,658,5,1678.0,S,4121,0,0,0,0,0,...,67,1968,7,1,0,55274,112695,35135,840,6
5,1401,0,16112.0,S,7011,0,0,0,0,0,...,71,1976,5,0,1,14214,28983,36482,755,3
6,1785,2,34005.0,S,7011,0,0,0,0,0,...,64,1936,1,1,1,41665,66745,682,718,3
7,1012,0,8932.0,S,7011,0,0,0,0,0,...,69,1975,5,1,0,26966,54981,121701,737,1
8,395,1,29041.0,Ch,7011,0,0,0,0,0,...,65,1951,12,0,1,32079,50184,27778,732,4
9,1753,4,22697.0,S,7011,0,0,0,0,0,...,65,1926,1,1,1,28147,40787,1574,782,6


In [ ]:
del df_users, df_cards
print(df_v2 .keys())

Index(['User', 'Card', 'Amount', 'Use_Chip', 'MCC', 'Is_Fraud?', 'Bad_PIN',
       'Technical_Glitch', 'Bad_Card_Number', 'Bad_CVV', 'Bad_Expiration',
       'Bad_Zipcode', 'Errors', 'Insufficient_Balance', 'InternationalT',
       'Timec', 'Date', 'TranYear', 'TranMonth', 'TranWeek', 'TranDay',
       'TranDayofweek', 'TranDayofyear', 'TranIs_month_end',
       'TranIs_month_start', 'TranIs_quarter_end', 'TranIs_quarter_start',
       'TranIs_year_end', 'TranIs_year_start', 'TranElapsed', 'Hour', 'hr_sin',
       'hr_cos', 'mnth_sin', 'mnth_cos', 'enc_MN', 'enc_MS', 'enc_UI',
       'distance', 'Card Brand', 'Card Type', 'Has Chip', 'Cards Issued',
       'Credit Limit', 'Expires_year', 'Expires_month', 'Acct Open Date_year',
       'Acct Open Date_month', 'frequent pin', 'frequecy change cardm',
       'Current Age', 'Retirement Age', 'Birth Year', 'Birth Month', 'Gender',
       'Apartment', 'Per Capita Income - Zipcode', 'Yearly Income - Person',
       'Total Debt', 'FICO Score', 

In [ ]:
df_v2 .keys()

Index(['User', 'Card', 'Amount', 'Use_Chip', 'MCC', 'Is_Fraud?', 'Bad_PIN',
       'Technical_Glitch', 'Bad_Card_Number', 'Bad_CVV', 'Bad_Expiration',
       'Bad_Zipcode', 'Errors', 'Insufficient_Balance', 'InternationalT',
       'Timec', 'Date', 'TranYear', 'TranMonth', 'TranWeek', 'TranDay',
       'TranDayofweek', 'TranDayofyear', 'TranIs_month_end',
       'TranIs_month_start', 'TranIs_quarter_end', 'TranIs_quarter_start',
       'TranIs_year_end', 'TranIs_year_start', 'TranElapsed', 'Hour', 'hr_sin',
       'hr_cos', 'mnth_sin', 'mnth_cos', 'enc_MN', 'enc_MS', 'enc_UI',
       'distance', 'Card Brand', 'Card Type', 'Has Chip', 'Cards Issued',
       'Credit Limit', 'Expires_year', 'Expires_month', 'Acct Open Date_year',
       'Acct Open Date_month', 'frequent pin', 'frequecy change cardm',
       'Current Age', 'Retirement Age', 'Birth Year', 'Birth Month', 'Gender',
       'Apartment', 'Per Capita Income - Zipcode', 'Yearly Income - Person',
       'Total Debt', 'FICO Score', 

In [ ]:
df_v2['Gender'].head()

0    1
1    1
2    1
3    1
4    1
Name: Gender, dtype: int64

In [ ]:
df_v2['Is_Fraud?'].value_counts()

0    24357143
1       29757
Name: Is_Fraud?, dtype: int64

In [ ]:
del df_v2['User']
del df_v2['Birth Month']

In [ ]:
for col in ['Gender', 'Timec']:
    df_v2[col] = df_v2[col].astype('category')

In [ ]:
df_v2['Has Chip'] = df_v2['Has Chip'].astype(int)

using categrory encoders for refrence https://contrib.scikit-learn.org/category_encoders/ 

In [ ]:
!pip install category_encoders
import category_encoders as ce
target = df_v2['Is_Fraud?']
del df_v2['Is_Fraud?']
cbe_encoder = ce.cat_boost.CatBoostEncoder()
cbe_encoder.fit(df_v2, target)
train_cbe = cbe_encoder.transform(df_v2)
df_v2=train_cbe
del train_cbe
df_v2['label']=target
del target
df_v2= pd.DataFrame(df_v2)
del df_v2['Date']
df_v2['label'].value_counts()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


CatBoostEncoder(cols=['Use_Chip', 'Timec', 'Card Brand', 'Card Type', 'Gender'])

using min max normalization ref https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html


In [ ]:
names=list(df_v2.columns)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df_v2)
df_v2 = pd.DataFrame(scaled_df, columns=names)
df_v2['label'].value_counts()

reducing datframe size by chaging coloumn types ref 
https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
df_v2 = reduce_memory_usage(df_v2, verbose=True)

Mem. usage decreased to 2697.83 Mb (75.0% reduction)


In [ ]:
df_v2['label'].value_counts()

0.0    24357143
1.0       29757
Name: label, dtype: int64

upload the file to google drive 

In [ ]:
df_v2.to_pickle('/content/drive/MyDrive/DataMining/file3' + '.pkl')